# CSE6242 - HW3 - Q1

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

Pyspark Imports

In [1]:
#export
### DO NOT MODIFY THIS CELL ###
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import hour, when, col, date_format, to_timestamp, ceil, coalesce

Initialize PySpark Context

In [2]:
### DO NOT MODIFY THIS CELL ###
sc = pyspark.SparkContext(appName="HW3-Q1")
sqlContext = SQLContext(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/18 21:19:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Define function for loading data

In [3]:
### DO NOT MODIFY THIS CELL ###
def load_data():
    df = sqlContext.read.option("header",True) \
     .csv("yellow_tripdata_2019-01_short.csv")
    return df

### Q1.1

Perform data casting to clean incoming dataset

In [4]:
#export
def clean_data(df):
    '''
    input: df a dataframe
    output: df a dataframe with the all the original columns
    '''
    
    # START YOUR CODE HERE ---------
    df.withColumn('passenger_count', df['passenger_count'].cast('int'))
    df.withColumn('total_amount', df['total_amount'].cast('float'))
    df.withColumn('tip_amount', df['tip_amount'].cast('float'))
    df.withColumn('trip_distance', df['trip_distance'].cast('float'))
    df.withColumn('fare_amount', df['fare_amount'].cast('float'))
    df.withColumn('tpep_pickup_datetime', df['tpep_pickup_datetime'].cast('timestamp'))
    df.withColumn('tpep_dropoff_datetime', df['tpep_dropoff_datetime'].cast('timestamp'))
  

    # END YOUR CODE HERE -----------
    
    return df

### Q1.2

Find rate per person for based on how many passengers travel between pickup and dropoff locations. 

In [5]:
#export
def common_pair(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - PULocationID
            - DOLocationID
            - total_passenger_count
            - per_person_rate
            
    per_person_rate is the total_amount per person for a given pair.
    
    '''
    
    # START YOUR CODE HERE ---------

    df = df.filter(col('PULocationID') != col('DOLocationID'))
    df = df.groupBy('PULocationID', 'DOLocationID').agg(
        {'passenger_count': 'sum', 'total_amount': 'sum'}
    )
    
    df = df.select(
        col('PULocationID'),
        col('DOLocationID'),
        col('sum(passenger_count)').cast('float').alias('total_passenger_count'),
        col('sum(total_amount)').cast('float').alias('total_amount')
    )
        
    df = df.withColumn('per_person_rate', col("total_amount") / col("total_passenger_count"))
    df = df.drop(col('total_amount'))

    df = df.orderBy(col('total_passenger_count').desc(), col('per_person_rate').desc())

    df = df.limit(10)
    
    # END YOUR CODE HERE -----------
    
    return df

### Q1.3

Find trips which trip distances generate the highest tip percentage.

In [23]:
#export
def distance_with_most_tip(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - trip_distance
            - tip_percent
            
    trip_percent is the percent of tip out of fare_amount
    
    '''
    
    # START YOUR CODE HERE ---------
    df = df.filter((col('trip_distance')>0) & (col('fare_amount')>2))
    df = df.withColumn('tip_percent', (col('tip_amount') * 100 / col('fare_amount')))
    df = df.withColumn('trip_distance', ceil(col('trip_distance')))

    df = df.groupBy('trip_distance').agg({'tip_percent':'mean'})
    df = df.withColumnRenamed('avg(tip_percent)', 'tip_percent')
    
    df = df.orderBy(col('tip_percent').desc()).limit(15)
    
    
    # END YOUR CODE HERE -----------
    
    return df

### Q1.4

Determine the average speed at different times of day.

In [ ]:
#export
def time_with_most_traffic(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - time_of_day
            - am_avg_speed
            - pm_avg_speed
            
    am_avg_speed and pm_avg_speed are the average trip distance / average trip time calculated for each hour
    
    '''
    
    # START YOUR CODE HERE ---------

    # END YOUR CODE HERE -----------
    
    return df

## The below cells are for you to investigate your solutions and will not be graded

In [12]:
df = load_data()
df = clean_data(df)

In [13]:
common_pair(df).show()

+------------+------------+---------------------+------------------+
|PULocationID|DOLocationID|total_passenger_count|   per_person_rate|
+------------+------------+---------------------+------------------+
|         239|         238|                 62.0|4.2627420733051915|
|         237|         236|                 60.0| 4.482500203450521|
|         263|         141|                 52.0| 3.419038332425631|
|         161|         236|                 42.0| 5.368571326846168|
|         148|          79|                 42.0| 4.711904616582961|
|         142|         238|                 39.0| 5.054871779221755|
|         141|         236|                 37.0| 4.355675774651605|
|         239|         143|                 37.0| 4.252162211650127|
|         239|         142|                 35.0| 3.817714146205357|
|          79|         170|                 34.0| 6.394705828498392|
+------------+------------+---------------------+------------------+



In [24]:
distance_with_most_tip(df).show()

+-------------+------------------+
|trip_distance|       tip_percent|
+-------------+------------------+
|            2|15.815527131655559|
|           17|15.796441782308916|
|           20| 15.11240992123345|
|            1|14.991505108704823|
|            3|14.886705733807549|
|            6|14.521284801943311|
|            5|14.245405873948084|
|            4|13.831569492912116|
|            9|13.814476585827178|
|            8| 12.07259674604865|
|           19|11.952632334985276|
|           10| 11.88049048422389|
|            7|10.800575589589778|
|           21|10.739019886973427|
|           18| 10.69682224321948|
+-------------+------------------+



In [ ]:
time_with_most_traffic(df).show()